# Notes on Python
## Examples for reference, tips, Best Practices

Based on the Course: Core Python (Metaclasses and Allocation) at PluralSight

Author: Gonçalo Felício  
Date: 04/2022  
Provided by: ISIWAY

Something like a pocketbook to come to for quick references, examples, and tips of best practices, compiled with my own preferences  
Loosely divided by subject, and with some degree, by the respective modules

### Allocation

The special method `__new__` allocates and returns a new instance  
This new instance is passed as the *self* argument to `__init__`  
Ultimately, it is `object.__new__(cls)` that is responsible for allocating new instances  
`__new__` is implicitly a static method, not requiring the static or class method decorators  
One use of implementing `__new__` is to support *Interning*. Interning is reusing existing objects of equal value, to vastly reduce memory consumption  
Interning should only be used with *imutable* objects!


In [2]:
class ChessBoard:

    def __new__(cls, *args, **kwargs): # just like a class method
        obj = object.__new__(cls)
        # to see info on __new__
        print(f'Class allocated = {cls.__name__}')
        print(f'Arguments = {args!r}')
        print(f'Key Arguments = {kwargs!r}')
        print(f'Id of object = {id(obj)}')
        return obj
    
    def __init__(self):
        
        print(f'Id of self = {id(self)}')
        
def main():
    test_class = ChessBoard()
    print('Clearly object and self are the same!')

if __name__ == '__main__':
    main()

Class allocated = ChessBoard
Arguments = ()
Key Arguments = {}
Id of object = 2561445709472
Id of self = 2561445709472
Clearly object and self are the same!


In [3]:
# tracing memory allocation
import tracemalloc
tracemalloc.start()
boards = [ChessBoard for _ in range(10000)]
_, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
peak_kb = peak / 10**3
print(f'Peak Kb = {peak_kb:.0f}')

Peak Kb = 12706


### Metaclasses
Metaclasses are the classes of classes  
In practice, the `type(class)` of a class is it's metaclass, and the default metaclass is `type`  
The metaclass is responsible for processing the class definition obtained by passing the source code, into a class object  
The `__prepare__` method must return a *namespace mapping* that is populated at runtime with namespace items collected from passing and executing the class definition  
The `__new__` method must allocate and return a class object that can be configured using the contents of the class namespace, the list of base classes and any aditional keyword arguments passed to its definition  
The `__init__` method can be used to configure a class object and must have the same signature as `__new__`  
The `__call__` method is the  true constructor for instances and is invoked when we construct instances  

In [15]:
class TracingMeta(type): #custom metaclasses should inherit from the base metaclass 'type'
    
    # mcs is the equivalent of cls in classes for metaclass methods, needs the decorator
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        print('TracingMeta.__prepare__(name, bases, **kwargs)')
        print(f' {mcs = }')
        print(f' {name = }')
        print(f' {bases = }')
        print(f' {kwargs = }')
        namespace = super().__prepare__(name, bases)
        print(f'-> {namespace = }')
        print()
        return namespace
    
    # __new__ is already, implicitly a classmethod, no decorator necessary
    def __new__(mcs, name, bases, namespace, **kwargs): 
        print('TracingMeta.__new__(name, bases, namespace, **kwargs)')
        print(f' {mcs = }')
        print(f' {name = }')
        print(f' {bases = }')
        print(f' {namespace = }')
        print(f' {kwargs = }')
        cls = super().__new__(mcs, name, bases, namespace)
        print(f'-> {cls = }')
        print()
        return cls

    def __init__(cls, name, bases, namespace, **kwargs):
        print('TracingMeta.__init__(cls, name, bases, namespace, kwargs)')
        print(f' {cls = }')
        print(f' {name = }')
        print(f' {bases = }')
        print(f' {namespace = }')
        print(f' {kwargs = }')
        super().__init__(name, bases, namespace)
        print()
    
    def __call__(cls, *args, **kwargs):
        print('TracingMeta.__call__(cls, args, kwargs)')
        print(f' {cls = }')
        print(f' {args = }')
        print('About to call type.__call__()')
        obj = super().__call__(*args, **kwargs)
        print('Returned from type.__call__()')
        print(f'-> {obj = }')
        print()
        return obj


In [16]:
class TracingClass(metaclass=TracingMeta):
    the_answer = 42
    def action(self, message):
        print(message)
    
    def __new__(cls, *args, **kwargs): 
        print('TracingClass.__new__(cls, *args **kwargs)')
        print(f' {cls = }')
        print(f' {args = }')
        print(f' {kwargs = }')
        obj = super().__new__(cls)
        print(f'-> {obj = }')
        print()
        return obj

    def __init__(self, *args, **kwargs):
        print('TracingClass.__init__(self, *args **kwargs)')
        print(f' {self = }')
        print(f' {args = }')
        print(f' {kwargs = }')
        print()

TracingMeta.__prepare__(name, bases, **kwargs)
 mcs = <class '__main__.TracingMeta'>
 name = 'TracingClass'
 bases = ()
 kwargs = {}
-> namespace = {}

TracingMeta.__new__(name, bases, namespace, **kwargs)
 mcs = <class '__main__.TracingMeta'>
 name = 'TracingClass'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'TracingClass', 'the_answer': 42, 'action': <function TracingClass.action at 0x0000025463B09F70>, '__new__': <function TracingClass.__new__ at 0x0000025463B09670>, '__init__': <function TracingClass.__init__ at 0x0000025463B095E0>, '__classcell__': <cell at 0x0000025463B26AF0: empty>}
 kwargs = {}
-> cls = <class '__main__.TracingClass'>

TracingMeta.__init__(cls, name, bases, namespace, kwargs)
 cls = <class '__main__.TracingClass'>
 name = 'TracingClass'
 bases = ()
 namespace = {'__module__': '__main__', '__qualname__': 'TracingClass', 'the_answer': 42, 'action': <function TracingClass.action at 0x0000025463B09F70>, '__new__': <function TracingClass.__ne

In [17]:
t = TracingClass(42, keyword='answer')

TracingMeta.__call__(cls, args, kwargs)
 cls = <class '__main__.TracingClass'>
 args = (42,)
About to call type.__call__()
TracingClass.__new__(cls, *args **kwargs)
 cls = <class '__main__.TracingClass'>
 args = (42,)
 kwargs = {'keyword': 'answer'}
-> obj = <__main__.TracingClass object at 0x000002546405F670>

TracingClass.__init__(self, *args **kwargs)
 self = <__main__.TracingClass object at 0x000002546405F670>
 args = (42,)
 kwargs = {'keyword': 'answer'}

Returned from type.__call__()
-> obj = <__main__.TracingClass object at 0x000002546405F670>



Subclass registration can be done by using the special method `__init__subclass`  
Metaclasses are inherited and there can only be *one* metaclass per class. This does not mean that a class can inherit from only one metaclass object. A new metaclass object can be constructed that it, itself, inherits directly from multiple metaclasses  

In [19]:
# check details of decoder example in module

from io import StringIO

class TableDecoder:
    _registry = {}
    
    @classmethod
    def __init_subclass__(cls, *, extension, **kwargs):
        super().__init_subclass(**kwargs)
        cls._registry[extension] = cls
    
    @classmethod
    def create(cls, name):
        decoder_class = cls._registry[name]
        return decoder_class()
    
    @classmethod
    def decoders(cls):
        return list(cls._registry.keys())
    
    def decode(self, text):
        raise NotImplementedError

Strict rules control how there multiple metaclasses inherited interact and care must be taken if they override the same methods. A safe course of action is to diligently apply the `super()` method.  Recall that the `super()` does not simlpy delegate to the Base class, but via the MRO, which contains all the subclasses related, allowing for metaclasses that compose correctly

In [21]:
class MetaA(type):
    pass

class MetaB(type):
    pass

class A(metaclass=MetaA):
    pass

class B(metaclass=MetaB):
    pass

class D(A):
    pass


In [22]:
type(D)

__main__.MetaA

In [23]:
class C(A, B):
    pass

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [24]:
class MetaC(MetaA, MetaB):
    pass

class C(A, B, metaclass=MetaC):
    pass

In [25]:
type(C)

__main__.MetaC